###Target:
Reach 99.4% accuracy with the power of image agumentation. Try to find the better Batch size for Mnist and this Architecture in 32, 64, 128, 256.

###Results:
1. Parameters: 7,917
2. Best Train Accuracy: 99.24%
3. Best Test Accuracy: 99.48%

###Analysis:
1. Image agumentation works as good regularization
2. 64 batch size showcases better performance than 32, 64, 128, 256
3. Model takes more time to converge with LR - 0.01 and was able to reaach > 99.4% accuracy only once in 14th epoch.
4. Adding more regularisation can make the model robust and perform consistently on test data.

 

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #block1
        self.conv1 = nn.Conv2d(1, 8, 3,padding = True) #28
        self.Batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 15, 3) #26
        self.Batchnorm2 = nn.BatchNorm2d(15)

        #transition block
        self.pool1 = nn.MaxPool2d(2, 2) #13
        self.pool1trns = nn.Conv2d(15, 10, 1)#13
        self.Batchnormtrns1 = nn.BatchNorm2d(10)
        
        #block2
        self.conv3 = nn.Conv2d(10,14,3) #11
        self.Batchnorm3 = nn.BatchNorm2d(14)
        self.conv4 = nn.Conv2d(14, 16, 3)#9
        self.Batchnorm4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 20, 3)#7
        self.Batchnorm5 = nn.BatchNorm2d(20)


        #block3
        #self.pool2trns = nn.Conv2d(18,10,1) #6
        #self.Batchnormtrns2 = nn.BatchNorm2d(10)
        #self.conv6 = nn.Conv2d(10,10,7) #1

        self.conv6_avgp = nn.AvgPool2d(kernel_size=7)

        self.pool2trns = nn.Conv2d(20,10,1) #6
        #self.Batchnormtrns2 = nn.BatchNorm2d(10)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.Batchnorm1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.Batchnorm2(x)


        x = self.pool1(x)
        x = self.pool1trns(x)
        x = F.relu(x)
        x = self.Batchnormtrns1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.Batchnorm3(x)

 
        x = self.conv4(x)
        x = F.relu(x)
        x = self.Batchnorm4(x)


        x = self.conv5(x)
        x = F.relu(x)
        x = self.Batchnorm5(x)


        x = self.conv6_avgp(x)

        x = self.pool2trns(x)
        #x = F.relu(x)
        #x = self.Batchnormtrns2(x)

        x = x.view(-1, 10)
        return F.log_softmax(x,dim=-1)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 15, 26, 26]           1,095
       BatchNorm2d-4           [-1, 15, 26, 26]              30
         MaxPool2d-5           [-1, 15, 13, 13]               0
            Conv2d-6           [-1, 10, 13, 13]             160
       BatchNorm2d-7           [-1, 10, 13, 13]              20
            Conv2d-8           [-1, 14, 11, 11]           1,274
       BatchNorm2d-9           [-1, 14, 11, 11]              28
           Conv2d-10             [-1, 16, 9, 9]           2,032
      BatchNorm2d-11             [-1, 16, 9, 9]              32
           Conv2d-12             [-1, 20, 7, 7]           2,900
      BatchNorm2d-13             [-1, 20, 7, 7]              40
        AvgPool2d-14             [-1, 2

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-8.0, 8.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    train_loss = 0 
    train_correct = 0
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(test_loader.dataset)
    print('Epoch: {:.0f},LR: {}.\nTrain set: train Average loss: {:.4f}, train_Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch,optimizer.param_groups[0]['lr'],train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
        

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred_test = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred_test.eq(target.view_as(pred_test)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    print('Test set: test Average loss: {:.4f}, test Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch: 1,LR: 0.01.
Train set: train Average loss: 1.9995, train_Accuracy: 54690/60000 (91.1500%)

Test set: test Average loss: 0.0658, test Accuracy: 9833/10000 (98.3300%)

Epoch: 2,LR: 0.01.
Train set: train Average loss: 0.4477, train_Accuracy: 58746/60000 (97.9100%)

Test set: test Average loss: 0.0390, test Accuracy: 9900/10000 (99.0000%)

Epoch: 3,LR: 0.01.
Train set: train Average loss: 0.3436, train_Accuracy: 58989/60000 (98.3150%)

Test set: test Average loss: 0.0330, test Accuracy: 9910/10000 (99.1000%)

Epoch: 4,LR: 0.01.
Train set: train Average loss: 0.2999, train_Accuracy: 59108/60000 (98.5133%)

Test set: test Average loss: 0.0316, test Accuracy: 9908/10000 (99.0800%)

Epoch: 5,LR: 0.01.
Train set: train Average loss: 0.2596, train_Accuracy: 59219/60000 (98.6983%)

Test set: test Average loss: 0.0286, test Accuracy: 9911/10000 (99.1100%)

Epoch: 6,LR: 0.01.
Train set: train Average loss: 0.2358, train_Accuracy: 59309/60000 (98.8483%)

Test set: test Average loss: 0.0266, 